Importamos las librerias necesarias

In [ ]:
import os
import time
import threading
import queue
import speech_recognition as sr

Definimos la funcion grabar_audio() que será la encargada de grabar el audio hasta que se detecte silencio. Ademas toma una muestra del sonido ambiente exterior para ajustar el posible ruido de la grabación.

La varibale q va a ser usadada como un buffer para comunicarse entre los hilos que va a haber en el prgrama.

In [ ]:
def grabar_audio(q):
    r = sr.Recognizer()

    with sr.Microphone() as source:
        while True:
            print("Escuchando...")
            #r.adjust_for_ambient_noise(source)
            audio = r.record(source, duration=5)  
            q.put(audio)

Definimos la funcion analizar audio que será la encargada de que cada vez que aparezcan datos de audio en la cola, los trasncirbirá haciendo uso de recognize_google(). Como en este proyecto vamos a enviarle audios en español debemos especificarle el lenguaje, ya que de otra manera no hará una trasncripción correcta.
La transcripcion será almacenada en un fichero de texto.

In [ ]:
def analizar_audio(q):
    r = sr.Recognizer()

    while True:
        audio = q.get()
        try:
            texto = r.recognize_google(audio, language="es-ES")
            print("Transcripción:", texto)
            with open("seq/transcripciones.txt", "a") as archivo:
                archivo.write(texto + "\n")

        except sr.UnknownValueError:
            print("No se pudo transcribir el audio")
        except sr.RequestError as e:
            print("Error en la solicitud al servicio de reconocimiento de voz:", str(e))

Finalmente definimos la función main que nos comprobará inicialmente si esta creado el directorio donde se almacenará el archivo de las trasncipciones, configurará la cola (q) para la comunicacion entre hilos y generará los hilos, uno para cada función creada anteriormente.

In [ ]:
if __name__ == "__main__":
    # Crea la carpeta 'seq' si no existe
    if not os.path.exists("seq"):
        os.makedirs("seq")

    # Configura la cola para compartir datos entre subprocesos
    cola = queue.Queue()

    # Crea los subprocesos
    hilo_grabacion = threading.Thread(target=grabar_audio, args=(cola,))
    hilo_analisis = threading.Thread(target=analizar_audio, args=(cola,))

    # Inicia los subprocesos
    hilo_grabacion.start()
    hilo_analisis.start()

    try:
        # Espera a que los subprocesos terminen
        hilo_grabacion.join()
        hilo_analisis.join()

    except KeyboardInterrupt:
        # Manejo de la interrupción de teclado (Ctrl+C)
        print("Programa interrumpido. Terminando subprocesos.")
        hilo_grabacion.join()
        hilo_analisis.join()